In [162]:
# basic python and ML Libraries
import os
import numpy as np
import pandas as pd
import pathlib

# reading images using OpenCV
import cv2

# matplotlib & others for visualization
import matplotlib.pyplot as plt

# torchvision libraries
import torch
import torchvision
from torchvision import transforms
from torch.utils.data import DataLoader, Dataset
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor

# for images
from PIL import Image, ImageDraw

device = "cuda" if torch.cuda.is_available() else "cpu"
root = pathlib.Path("data") / "car" / 'data'

In [163]:
# functions
def tensorprint(tensor):
    print("Shape: " , tensor.shape, " , Dimension: ", tensor.ndim , " \nDtype: ", tensor.dtype, " , Device: ", tensor.device)
    print("Max: ", tensor.amax(),f'[{tensor.argmax()}]', " , Min: ", tensor.amin(),f'[{tensor.argmin()}]')
    #print(tensor ,'\n')

def torch_rng():
    torch.manual_seed(42)
    torch.cuda.manual_seed(42)

def walk_through_dir(dir_path):
    """Walks Through dir_path returning its contents."""
    for dirpath, dirnames, filenames in os.walk(dir_path):
        print(f"directories: {len(dirnames)} | images: {len(filenames)} -> {dirpath}")

def collate_fn(batch):
    return tuple(zip(*batch))

def show_bbox(image_path, label):
    image = Image.open(image_path)
    draw = ImageDraw.Draw(image)

    x1, y1, x2, y2 = label[0], label[1], label[2], label[3]
    draw.rectangle((x1,y1,x2,y2), outline = (255,0,0), width = 5)
    image.show()

def OD_train_step(model, optimizer, data_loader, device = device):
    model.train()
    train_loss_list = []
    c = 0

    for images , targets in data_loader:
        images = list(image.to(device) for image in images)
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]
        
        loss_dict = model(images, targets)
        losses = sum(loss for loss in loss_dict.values())
        loss_value = losses.item()
        train_loss_list.append(loss_value)


        optimizer.zero_grad()
        losses.backward()
        optimizer.step()
        c+=1

        if c % 4 == 0:
            print(f"Training loss: {loss_value}")

    return train_loss_list

def OD_test_step(model, optimizer, data_loader, device = device):
    val_loss_list = []
    c= 0

    for images, targets in data_loader:
        images = list(image.to(device) for image in images)
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

        with torch.inference_mode():
            loss_dict = model(images, targets)
            
            losses = sum(loss for loss in loss_dict.values())
            loss_value = losses.item()
            val_loss_list.append(loss_value)
            c+=1

            if c % 4 == 0:
                print(f"Testing loss: {loss_value}")

        return val_loss_list

def get_transform(img, target):
    train_transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),                        
    ])

    return train_transform(img), target

In [164]:
# setup 
train_dir = root / 'training_images'
test_dir = root / 'testing_images'
train_label_dir = root / 'train_solution_bounding_boxes (1).csv'

train_image_list = sorted(os.listdir(train_dir))
test_image_list = sorted(os.listdir(test_dir))

In [167]:
# dataset 

class CarDataset(torch.utils.data.Dataset):
    def __init__(self,image, label, transforms) -> None:
        self.image = image
        self.label = label
        self.transforms = transforms

        self.imgs = os.listdir(image)
        self.labels = pd.read_csv(label)

        self.label_image_check = self.labels[self.labels['image'].isin(self.imgs)]
        self.labels_name_list = self.label_image_check['image'].tolist()

        self.imgs = [i for i in self.labels_name_list if i in self.imgs]

        check = True if len(e.imgs) == len(e.labels) else False

    def __getitem__(self,idx):
        W, H = 640, 640
        img_path = os.path.join(self.image , self.imgs[idx])
        labels = self.labels[self.labels['image'] == self.imgs[idx]]
        img = Image.open(img_path)
        img = img.resize((W,H), Image.LANCZOS)
        
        boxes = labels.drop(columns=['image'], axis=1).values.tolist()[0]
        labels = 1

        boxes = torch.as_tensor(boxes, dtype=torch.float32)
        labels = torch.as_tensor(labels, dtype=torch.int64)

        area = ((boxes[3] - boxes[1]) * (boxes[2] - boxes[0]))
        iscrowd = torch.zeros(boxes.shape[0], dtype = torch.int64)

        target = {}
        target['boxes'] = boxes
        target['labels'] = labels
        target['area'] = area
        target['iscrowd'] = iscrowd

        if self.transforms is not None:
            img, target = self.transforms(img, target)


        return img, target
    def __len__(self):
        return len(self.imgs)
    
e = CarDataset(train_dir, train_label_dir, None)
e[0]


(<PIL.Image.Image image mode=RGB size=640x640>,
 {'boxes': tensor([281.2590, 187.0351, 327.7279, 223.2255]),
  'labels': tensor(1),
  'area': tensor(1681.7317),
  'iscrowd': tensor([0, 0, 0, 0])})

In [168]:
# data loading
BATCH_SIZE = 32

train_data = CarDataset(train_dir, train_label_dir,get_transform)                             

train_dataloader = DataLoader(dataset=train_data, 
                                batch_size=BATCH_SIZE,
                                shuffle=True,
                                collate_fn=collate_fn)

                                
img, target = next(iter(train_dataloader))
images = list(image.to(device) for image in img)
targets = [{k : v.to(device) for k , v in t.items()} for t in target]


In [ ]:
# model
